<a href="https://colab.research.google.com/github/mnmahir/best-fyp/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Libraries**

In [ ]:
# Google Drive
from google.colab import drive
# General
import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import itertools
import random as python_random
# Keras Library
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPool2D
from keras.models import Model
from keras import backend as Kbackend
from keras.preprocessing import image
# ScikitLearn
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
# Others
import time
import datetime
import os

print("Keras version: ", keras.__version__)
print("TensorFlow version:", tf.__version__)

In [ ]:
# Check GPU. With Colab Pro you have priority access to our fastest GPUs. For example, you may get a T4 or P100 GPU at times when most users of standard Colab receive a slower K80 GPU. You can see what GPU you've been assigned at any time by executing the following cell.
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
# In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator dropdown to GPU.

# **Initialize Functions**

# Tools

In [ ]:
############################
# Solving class imbalance
############################
def get_freq_data(generator):
  class_data_labels = generator.classes
  total_class = len(IMAGE_CLASSES)
  freq_out = []

  for i in range(total_class):
    total_label_i = 0
    for ele in class_data_labels:
      if ele == i:
        total_label_i += 1
    freq_out.append(total_label_i)
  
  print("Total number of class is:", total_class, generator.class_indices)
  print("Total image in generator is: ", freq_out)
  return freq_out
############################
# Display training and validation curve
############################
def display_training_curves(training, validation, title, subplot):
  ax = plt.subplot(subplot)
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('Model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['Training', 'Validation'])

############################
# Display confusion matrix
############################
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues, subplot = 121):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        cm = np.round(cm,2)
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.subplot(subplot)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))  # Count number of classes for x & y axis lable
    plt.xticks(tick_marks, classes, rotation=45) # x-axis label rotated 45 degree
    plt.yticks(tick_marks, classes) # y-axis label
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
  
############################
# Show generated image
############################
def show_image_generated(generator, index, img_size):
  x, y = generator.__getitem__(index)
  img_plt = math.ceil(math.sqrt(BATCH_SIZE))
  grid_plt = str(img_plt)+str(img_plt)+str(1)
  plt.figure(1, figsize = (img_size,img_size))
  for i in range(BATCH_SIZE):
    plt.subplot(img_plt,img_plt,i+1)
    plt.imshow(x[i])
    plt.title(IMAGE_CLASSES[np.where(y[i] == 1.)[0][0]], fontsize = 2+img_size, pad = 0)
    plt.axis('off')
  plt.show()

###########################
# Show augmented image
###########################
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plot_augmented_images(img_generator, num, index_i = 0, index_j = 0):
    images_arr = [img_generator[index_i][0][index_j] for i in range(num)]
    fig, axes = plt.subplots(1, num, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.axis('off')
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

############################
# Show all prediction images
############################
def show_all_prediction_image(class_dir, generator, true_labels, pred_labels):
  image_freq = get_freq_data(generator)
  correct_pred = []
  wrong_pred = []
  for i in range(len(IMAGE_CLASSES)):
    correct_num = 0
    wrong_num = 0
    for j in range(len(true_labels)):
      if true_labels[j] == i:
        if true_labels[j] == pred_labels[j]:
          correct_num += 1
        else:
          wrong_num += 1
    correct_pred.append(correct_num)
    wrong_pred.append(wrong_num)

  print('Correctly predicted:',correct_pred)
  print('Wrongly predicted:',wrong_pred)

  label_index_show = 0

  disp_col = 8

  for i, class_f in enumerate(os.listdir(class_dir)):
    img_dir = class_dir + '/' + class_f
    display_row = math.ceil(image_freq[i]/disp_col)
    batch_holder = np.zeros((image_freq[i], IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    print('========================================================================')
    print('True Class:', class_f)
    print('Total images:', image_freq[i],'| Correctly predicted:',correct_pred[i],'| Wrongly predicted:',wrong_pred[i])
    print('========================================================================')

    for j,img in enumerate(os.listdir(img_dir)):
      img = image.load_img(os.path.join(img_dir,img), target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))
      batch_holder[j, :] = img

    plt.figure(1, figsize = (disp_col*1.8,display_row*2.1))

    for j,img in enumerate(batch_holder):
      title = 'T: ' + IMAGE_CLASSES[true_labels[label_index_show]] + '\nP: ' + IMAGE_CLASSES[pred_labels[label_index_show]]
      plt.subplot(display_row, disp_col, j+1)
      if true_labels[label_index_show] == pred_labels[label_index_show]:
        plt.title(title)
      else:
        plt.title(title,color = 'red')
      plt.axis('off')
      plt.imshow(img/256.)
      label_index_show += 1
      
    plt.show()

############################
# Get input image for pre-trained model
############################
def get_pretrained_model_input_size(MODEL = 'None'):
  if MODEL == 'Xception' or MODEL == 'InceptionV3' or MODEL == 'InceptionResNetV2':
    return (299, 299, 3)
  elif MODEL == 'VGG16' or MODEL == 'VGG19' or MODEL == 'ResNet50V2' or MODEL == 'ResNet101V2' or MODEL == 'ResNet152V2' or MODEL == 'MobileNetV2' or MODEL == 'DenseNet121' or MODEL == 'DenseNet169' or MODEL == 'DenseNet201' or MODEL == 'NASNetMobile':
    return (224, 224, 3)
  elif MODEL == 'NASNetLarge':
    return (331, 331, 3)
  else:
    print("Invalid model name/ not defined")
    return (0,0,0)

# Data Augmentation

In [ ]:
############################
# Training data generator
############################
def get_train_generator():
  print("Getting train generator...")
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 90,
      vertical_flip = True,
      horizontal_flip = True,
      fill_mode = 'reflect'
  )

  generator = image_generator.flow_from_directory(
      batch_size = BATCH_SIZE,
      directory = train_path,
      shuffle = True,             # Shuffle input after each epoch
      target_size = (IMAGE_WIDTH, IMAGE_HEIGHT),   # Set image size to be [x]px by [y]px set in IMAGE_SIZE
      class_mode = 'categorical',
      seed = SEED,
  )

  return generator
############################
# Validation data generator
############################
def get_valid_generator():
  print("Getting valid generator...")
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255
  )

  generator = image_generator.flow_from_directory(
      batch_size = BATCH_SIZE,
      directory = valid_path,
      shuffle = False,             # Shuffle input after each epoch
      target_size = (IMAGE_WIDTH, IMAGE_HEIGHT),   # Set image size to be [x]px by [y]px set in IMAGE_SIZE
      class_mode = 'categorical',
      seed = SEED,
  )

  return generator
############################
# Test data generator
############################
def get_test_generator():
  print("Getting test generator...")
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255
  )

  generator = image_generator.flow_from_directory(
      batch_size = BATCH_SIZE,
      directory = test_path,
      shuffle = False,             # Shuffle input after each epoch
      target_size = (IMAGE_WIDTH, IMAGE_HEIGHT),   # Set image size to be [x]px by [y]px set in IMAGE_SIZE
      class_mode = 'categorical',
      seed = SEED,
  )

  return generator

# Model Development

In [ ]:
############################
# Custom model
############################
def get_custom_model(print_summary = False):
  model = Sequential()
  model.add(Convolution2D(nb_filters, (5, 5), activation = 'relu', padding='same', input_shape=input_shape))
  model.add(Convolution2D(nb_filters, (5, 5), activation = 'relu', padding='same'))
  model.add(MaxPool2D(pool_size=pool_size))
  model.add(Dropout(rate=0.25))
  model.add(Convolution2D(nb_filters*2, (3, 3), activation='relu', padding='same'))
  model.add(Convolution2D(nb_filters*2, (3, 3), activation='relu', padding='same'))
  model.add(Convolution2D(nb_filters*2, (3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=pool_size))
  model.add(Dropout(rate=0.25))
  model.add(Convolution2D(nb_filters*2, (3, 3), activation='relu', padding='same'))
  model.add(Convolution2D(nb_filters*2, (3, 3), activation='relu', padding='same'))
  model.add(MaxPool2D(pool_size=pool_size))
  model.add(Dropout(rate=0.25))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  if print_summary:
    model.summary()
  return model

In [ ]:
############################
# Transfer learning model
############################
def get_pretrained_model(transfer_learning_model = 'None', weights = None, unfreeze_last_layer = 0, print_summary = False, print_fine_tuning_summary = True):
  ################ Choose Pre-trained model #########################
  if transfer_learning_model == 'Xception':
    selected_model = "Xception pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.Xception(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)
  
  elif transfer_learning_model == 'VGG16':
    selected_model = "VGG16 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.VGG16(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)
  
  elif transfer_learning_model == 'VGG19':
    selected_model = "VGG19 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.VGG19(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'ResNet50V2':
    selected_model = "ResNet50V2 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.ResNet50V2(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)
  
  elif transfer_learning_model == 'ResNet101V2':
    selected_model = "ResNet101V2 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.ResNet101V2(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'ResNet152V2':
    selected_model = "ResNet152V2 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.ResNet152V2(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'InceptionV3':
    selected_model = "InceptionV3 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.InceptionV3(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'InceptionResNetV2':
    selected_model = "InceptionResNetV2 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.InceptionResNetV2(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'MobileNetV2':
    selected_model = "MobileNetV2 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.MobileNetV2(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'DenseNet121':
    selected_model = "DenseNet121 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.DenseNet121(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'DenseNet169':
    selected_model = "DenseNet169 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.DenseNet169(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'DenseNet201':
    selected_model = "DenseNet201 pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.DenseNet201(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'NASNetMobile':
    selected_model = "NASNetMobile pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.NASNetMobile(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)

  elif transfer_learning_model == 'NASNetLarge':
    selected_model = "NASNetLarge pre-trained model selected"
    print(selected_model)
    pretrained_model = tf.keras.applications.NASNetLarge(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL), weights = weights, include_top=False)
  
  else:
    print("Not defined / Does not exist")
    return 0
  ##################################################################
  # Freeze early layers
  for layer in pretrained_model.layers:
    if unfreeze_last_layer < len(pretrained_model.layers):
      layer.trainable = False
      unfreeze_last_layer += 1

  # Print summary of pre-trained model
  if print_summary:
    pretrained_model.summary()
    if unfreeze_last_layer > 0:
      print("####################### UNFREEZED LAYER #########################")
      print("Layer            Trainable")
      for layer in pretrained_model.layers:
        if layer.trainable == True:
          sp = '      '[len(layer.name)-9:]
          print(layer.name, sp, layer.trainable)

  model = Sequential()
  model.add(pretrained_model)

  # Fine tuning our layers - you can add more if you want
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(256, activation = 'relu'))
  model.add(Dropout(0.2))

  # Prediction layer
  model.add(layers.Dense(len(IMAGE_CLASSES), activation = 'softmax'))

  # Compile model
  model.compile(
      optimizer='adam',
      loss = 'categorical_crossentropy',
      metrics=['accuracy']
  )

  if print_fine_tuning_summary:
    print("###################### AFTER FINE TUNING ########################")
    model.summary()
    print(selected_model)

  return model


# **Training**

In [ ]:
# Path where dataset is located
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/FYP Stuff/Datasets/'
dataset_name = 'Dataset 400A'
train_path = dataset_path + dataset_name + '/Train'
valid_path = dataset_path + dataset_name + '/Valid'    # Some literature use the term "test set" / "valid set" but it is actually a "dev set". This is the set that is iterated during training.
test_path = dataset_path + dataset_name + '/Valid'  # Test set helps evaluate how good your final system is. It's ok not to have test set.

In [ ]:
IMAGE_CLASSES = ['Normal','Grade I','Grade II','Grade III']         # Naming Classes (The names must be in the same order as in dataset folder (usually alphabetically))
MODEL = 'Xception'              # Xception, VGG16, VGG19, InceptionResNetV2, ResNet50V2, ResNet101V2, ResNet152V2, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet121, DenseNet169, DenseNet201, NASNetMobile, NASNetLarge
SAVE_MODEL_NAME = MODEL + '_' + dataset_name + '_awesome_model.h5'         # Name of the model to save after training
BATCH_SIZE = 64                 # Mini-batch size
SEED = 2020                     # Fixed randomness (For reproducible result!)
python_random.seed(SEED)        # necessary for starting core Python generated random numbers in a well-defined state.
np.random.seed(SEED)            # necessary for starting Numpy generated random numbers in a well-defined state.
tf.random.set_seed(SEED)        # will make random number generation in the TensorFlow backend have a well-defined initial state.

IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL = get_pretrained_model_input_size(MODEL)
print("Training on", MODEL)
print("Input:",IMAGE_WIDTH,'x',IMAGE_HEIGHT,'x',IMAGE_CHANNEL)

In [ ]:
train_generator = get_train_generator()
valid_generator = get_valid_generator()
test_generator = get_test_generator()

In [ ]:
# Show image generated by generator
# show_image_generated(train_generator, index = 3, img_size = 12)  # Print image generated by generator. Change index to see different batch (start at 0)
# plot_augmented_images(train_generator,5,0,0)

In [ ]:
# Check number of image in each class
train_freq = get_freq_data(train_generator)
valid_freq = get_freq_data(valid_generator)
total_freq = np.array(train_freq) + np.array(valid_freq)
valid_data_freq = np.array(valid_freq)/total_freq
print("Valid data Percentage:",valid_data_freq)



In [ ]:
model = get_pretrained_model(MODEL, weights = 'imagenet', unfreeze_last_layer = 0, print_summary = False, print_fine_tuning_summary = True)

In [ ]:
EPOCH = 50                      # Number of training epoch

time_start = time.time()
history = model.fit(
    x = train_generator,
    validation_data = valid_generator,
    epochs = EPOCH,
    verbose = 1,
    steps_per_epoch = train_generator.samples / train_generator.batch_size ,
    validation_steps = valid_generator.samples / valid_generator.batch_size
)
print("Training done. Evaluating...")
time_end = time.time()
final_accuracy = history.history["val_accuracy"][-5:]
Training_Accuracy = model.evaluate(train_generator, steps=24)
Validation_Accuracy = model.evaluate(test_generator, steps=24)
print("Finish!")

In [ ]:
print("Trained on", MODEL)
print("Time taken:", datetime.timedelta(seconds=(int(time_end - time_start))), "(hh:mm:ss)")
print("The Model Accuracy for Train data is: ", "{:.2%}".format(Training_Accuracy[1]))
print("The Model Accuracy for Validation data is: ", "{:.2%}".format(Validation_Accuracy[1]))
print("FINAL VALIDATION ACCURACY MEAN-5: ", np.mean(final_accuracy))
# Loss and Accuracy
plt.subplots(figsize=(9,3.5))
plt.tight_layout()
display_training_curves(history.history['loss'], history.history['val_loss'], 'Loss', 121)
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'Accuracy', 122)

# **Save & Reload Model**

In [ ]:
model.save(SAVE_MODEL_NAME) #Save
#reloaded_model = tf.keras.models.load_model(SAVE_MODEL_NAME) #
reloaded_model = tf.keras.models.load_model('awesome_model.h5') #Load

# **Evaluation**

**Predict**

In [ ]:
# True label
true_labels = test_generator.classes  # Get label of valid_generator
# Prediction label
predictions = reloaded_model.predict(x=test_generator, steps=len(test_generator),verbose = 0)
pred_labels = np.argmax(predictions, axis=-1)

**Confusion Matrix**

In [ ]:
# Plot Confusion Matrix
cm = confusion_matrix(y_true=true_labels, y_pred=pred_labels)
plt.subplots(figsize=(10,4))
plot_confusion_matrix(cm=cm, classes=IMAGE_CLASSES, title='Confusion Matrix',normalize=False, subplot=121)
plot_confusion_matrix(cm=cm, classes=IMAGE_CLASSES, title='Normalized Confusion Matrix',normalize=True, subplot=122)

**Other Metrics**

In [ ]:
report = metrics.classification_report(y_true = true_labels, y_pred = pred_labels, target_names=IMAGE_CLASSES)
print(report)

**Show all images with predictions**

In [ ]:
show_all_prediction_image(test_path, test_generator, true_labels, pred_labels) # Argument:(Path of test dataset, test generator to generate image, true labels generated by test generator, predicted labels generated by model on test dataset)